**Sentiment Analysis of Movie Reviews Using Support Vector
Machine**

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Step 1 : Data Collection & Cleaning

In [ ]:
raw=load_dataset("imdb")
df = pd.concat([pd.DataFrame(raw["train"]),
                pd.DataFrame(raw["test"])],ignore_index=True)
df.head()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [ ]:
df["text"] = df["text"].fillna("")

In [ ]:
stop_words = set(stopwords.words("english"))
stop_words.discard("not")
stop_words.discard("no")
stop_words.discard("nor")

lemmatizer = WordNetLemmatizer()
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(w)
        for w in tokens
        if w not in stop_words
    ]
    return " ".join(tokens)

In [ ]:
df["clean_text"] = df["text"].apply(preprocess)
df[["text", "clean_text"]].head()

,text,clean_text
0,I rented I AM CURIOUS-YELLOW from my video sto...,rented curiousyellow video store controversy s...
1,"""I Am Curious: Yellow"" is a risible and preten...",curious yellow risible pretentious steaming pi...
2,If only to avoid making this type of film in t...,avoid making type film future film interesting...
3,This film was probably inspired by Godard's Ma...,film probably inspired godard masculin fminin ...
4,"Oh, brother...after hearing about this ridicul...",oh brotherafter hearing ridiculous film umptee...


In [ ]:
X = df["clean_text"]
y = df["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)

Step 2:Feature Extraction using TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000,ngram_range=(1, 2),sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf  = vectorizer.transform(X_test)

Step 3 : Model Training

In [ ]:
svm_linear = SVC(kernel="linear", C=1, class_weight="balanced")
svm_poly = SVC(kernel="poly", degree=3, C=1)
svm_rbf = SVC(kernel="rbf", gamma="scale", C=1)
svm_linear.fit(X_train_tfidf, y_train)
svm_poly.fit(X_train_tfidf, y_train)
svm_rbf.fit(X_train_tfidf, y_train)

SVC(C=1)

In [ ]:
def evaluate(model, X, y, name):
    y_pred = model.predict(X)
    acc = accuracy_score(y, y_pred)
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y, y_pred))
    return acc

Step 4 : Model Comparision

In [ ]:
acc_linear = evaluate(svm_linear, X_test_tfidf, y_test, "Linear SVM")
acc_poly   = evaluate(svm_poly, X_test_tfidf, y_test, "Polynomial SVM")
acc_rbf    = evaluate(svm_rbf, X_test_tfidf, y_test, "RBF SVM")


Linear SVM Accuracy: 0.9007
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      5000
           1       0.90      0.91      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000


Polynomial SVM Accuracy: 0.8749
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      5000
           1       0.84      0.92      0.88      5000

    accuracy                           0.87     10000
   macro avg       0.88      0.87      0.87     10000
weighted avg       0.88      0.87      0.87     10000


RBF SVM Accuracy: 0.9054
              precision    recall  f1-score   support

           0       0.91      0.90      0.90      5000
           1       0.90      0.91      0.91      5000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0

In [ ]:
results = pd.DataFrame({"Kernel": ["Linear", "Polynomial", "RBF"],
                        "Accuracy": [acc_linear, acc_poly, acc_rbf]})
results

,Kernel,Accuracy
0,Linear,0.9007
1,Polynomial,0.8749
2,RBF,0.9054


In [ ]:
def predict_sentiment(text):
    clean = preprocess(text)
    vec = vectorizer.transform([clean])
    pred = svm_linear.predict(vec)[0]
    return "Positive" if pred == 1 else "Negative"


Testing from user input

In [ ]:
sentence = input("Enter a review: ")
print("Sentiment:", predict_sentiment(sentence))


Enter a review: thats a crazy stunt he did 
Sentiment: Positive
